In [1]:
# @title Show job status
#!pip install paramiko

import ipywidgets as widgets
from IPython.display import HTML, display, Javascript
import getpass
import paramiko
import stat
import os
import time as timer
import subprocess
from urllib.parse import parse_qs
import sys
from urllib.parse import urlparse, parse_qs
import requests

hostname =""
username =""
job_id =""
pkey = ""
%store -r hostname
%store -r username
%store -r job_id
%store -r pkey

cluster_input = widgets.Text(value=hostname, description="Cluster:")
username_input = widgets.Text(value=username, description="User name:")
user_password = widgets.Password(placeholder='Enter password',
    description='Password:',
    disabled=False
)
user_pkey = widgets.Text(value=pkey, placeholder='Enter path to private key',
    description='Private key:',
    disabled=False
)
job_id_input = widgets.Text(value=job_id, description="Enter job id:")
submit_button = widgets.Button(description="View Job status/output")

input_accordion = widgets.Accordion(children=[widgets.VBox([cluster_input, username_input, user_password, user_pkey, job_id_input, submit_button])])
input_accordion.set_title(0, 'Enter details to see job status/output')
display(input_accordion)

# Output
output = widgets.Output()
display(output)

# Function to be executed on button click
@output.capture()
def on_submit_button_click(button):
  hostname = cluster_input.value
  username = username_input.value
  password = user_password.value
  pkey = user_pkey.value
  job_id = job_id_input.value
  ssh_client = paramiko.SSHClient()
  ssh_client.load_system_host_keys()
  ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
  try:
    if pkey != '':
        print("Using private key provided")
        key = paramiko.pkey.PKey.from_path(pkey)
        ssh_client.connect(hostname, username=username, pkey=key, allow_agent=True)
    elif password != '':
        print("Using password provided")
        ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
  except:
      print("Unsuccessful authentication")
      pass
  transport = ssh_client.get_transport()
  #transport.auth_password(username, getpass.getpass('Enter {0} Logon password :'.format(hostname)))
  sftp_client = paramiko.SFTPClient.from_transport(transport)


  # job status
  # Check the job status periodically
  while True:
    _, stdo, stde = ssh_client.exec_command(f"squeue -u {username} -j {job_id}")
    job_status = stdo.read().decode()
    print(job_status)

    # Break the loop if the job is completed or failed
    if job_id not in job_status:
        break

    # Wait for a few seconds before checking again
    timer.sleep(10)

  print("Output")
  # View output
  _, stdo, stde = ssh_client.exec_command(f"ls -l")
  print(stdo.read().decode())
  print(stde.read().decode())

  print("Output")
  # View output
  _, stdo, stde = ssh_client.exec_command(f"cat *.o{job_id}")
  print(stdo.read().decode())
  print(stde.read().decode())

  print("Error")
  # View error
  _, stdo, stde = ssh_client.exec_command(f"cat *.e{job_id}")
  print(stdo.read().decode())
  print(stde.read().decode())


  sftp_client.close()
  ssh_client.close()
  transport.close()

# Attach the function to the button click event
submit_button.on_click(on_submit_button_click)

no stored variable or alias hostname
no stored variable or alias username
no stored variable or alias job_id
no stored variable or alias pkey


Accordion(children=(VBox(children=(Text(value='', description='Cluster:'), Text(value='', description='User na…

Output()